#Зчитати таблиці з папки silver

In [0]:
import os

# Функція для створення директорії
def ensure_dir_exists(path):
    os.makedirs(path, exist_ok=True)

In [0]:
# Шляхи до папки bronze і silver
gold_path = "/tmp/gold/"
silver_path = "/tmp/silver/"

ensure_dir_exists(gold_path)

In [0]:
# Зчитування таблиць
input_path = f"{silver_path}athlete_bio"
athlete_bio_df = spark.read.parquet(input_path)

input_path = f"{silver_path}athlete_event_results"
athlete_event_results_df = spark.read.parquet(input_path)

#Об'єднати таблиці за колонкою athlete_id

In [0]:
athlete_event_results_df = athlete_event_results_df.drop("country_noc")

joined_df = athlete_bio_df.join(athlete_event_results_df, on="athlete_id", how="inner")
joined_df.printSchema()

root
 |-- athlete_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- born: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- description: string (nullable = true)
 |-- special_notes: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- edition_id: integer (nullable = true)
 |-- sport: string (nullable = true)
 |-- event: string (nullable = true)
 |-- result_id: integer (nullable = true)
 |-- athlete: string (nullable = true)
 |-- pos: string (nullable = true)
 |-- medal: string (nullable = true)
 |-- isTeamSport: boolean (nullable = true)



#знаходження середнього значення weight і height для комбінації sport, medal, sex, country_noc

In [0]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import round

# Групування за комбінацією стовпчиків та обчислення середніх значень weight і height
aggregated_df = (
    joined_df.groupBy("sport", "medal", "sex", "country_noc")
    .agg(
        round(avg("weight"), 2).alias("avg_weight"),  # Округлення до 2 знаків
        round(avg("height"), 2).alias("avg_height")  # Округлення до 2 знаків
    )
)

aggregated_df.printSchema()

root
 |-- sport: string (nullable = true)
 |-- medal: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- avg_weight: double (nullable = true)
 |-- avg_height: double (nullable = true)



#Додати колонку з часовою міткою виконання програми (timestamp)

In [0]:
from pyspark.sql.functions import current_timestamp

# Додавання колонки з часовою міткою
avg_stats = aggregated_df.withColumn("timestamp", current_timestamp())

# Перегляд результатів
avg_stats.printSchema(10)

root
 |-- sport: string (nullable = true)
 |-- medal: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- avg_weight: double (nullable = true)
 |-- avg_height: double (nullable = true)
 |-- timestamp: timestamp (nullable = false)



#Запис даних в gold/avg_stats

In [0]:
    table = 'avg_stats'
    output_path = f"{gold_path}{table}"
    
    # Перевірка наявності директорії
    ensure_dir_exists(output_path)
    
    # Збереження у форматі Parquet
    avg_stats.write.mode("overwrite").parquet(output_path)
    print(f"Таблиця збережена в: {output_path}")

    # Перепровірка наявності записаного датасету
    df = spark.read.parquet(output_path)
    df.show(truncate=False)

Таблиця збережена в: /tmp/gold/avg_stats
+------------------+------+------+-----------+----------+----------+-----------------------+
|sport             |medal |sex   |country_noc|avg_weight|avg_height|timestamp              |
+------------------+------+------+-----------+----------+----------+-----------------------+
|IceHockey         |None  |Male  |FIN        |4177.51   |180.79    |2024-12-22 22:03:47.814|
|Rowing            |None  |Female|CAN        |67.62     |174.71    |2024-12-22 22:03:47.814|
|Wrestling         |None  |Male  |DEN        |68.32     |169.86    |2024-12-22 22:03:47.814|
|Fencing           |None  |Male  |POL        |74.17     |179.03    |2024-12-22 22:03:47.814|
|Football          |None  |Female|BRA        |59.87     |166.86    |2024-12-22 22:03:47.814|
|Badminton         |None  |Male  |THA        |68.52     |174.96    |2024-12-22 22:03:47.814|
|Athletics         |None  |Male  |ANG        |65.92     |175.52    |2024-12-22 22:03:47.814|
|CrossCountrySkiing|None  |Fe